In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
import pickle

from google.cloud import storage
from google.colab import auth
from apiclient.http import MediaIoBaseDownload
from googleapiclient.discovery import build
auth.authenticate_user()
gcs_service = build('storage', 'v1')

In [2]:
#read dataset csv from bucket
with open('/tmp/gsutil_download.csv', 'wb') as f:
  # Download the file from a given Google Cloud Storage bucket.
  request = gcs_service.objects().get_media(bucket='malmem-dataset',
                                            object='malmem_dataset.csv')
  media = MediaIoBaseDownload(f, request)

  done = False
  while not done:
    _, done = media.next_chunk()

df = pd.read_csv('/tmp/gsutil_download.csv')



In [3]:
X = df.copy()
X = X.sort_index(axis=1)
X

,Category,Class,callbacks.nanonymous,callbacks.ncallbacks,callbacks.ngeneric,dlllist.avg_dlls_per_proc,dlllist.ndlls,handles.avg_handles_per_proc,handles.ndesktop,handles.ndirectory,...,psxview.not_in_pspcid_list_false_avg,psxview.not_in_session,psxview.not_in_session_false_avg,svcscan.fs_drivers,svcscan.interactive_process_services,svcscan.kernel_drivers,svcscan.nactive,svcscan.nservices,svcscan.process_services,svcscan.shared_process_services
0,Benign,Benign,0,87,8,38.500000,1694,212.302326,46,104,...,0.042553,4,0.085106,26,0,221,121,389,24,116
1,Benign,Benign,0,87,8,44.127660,2074,242.234043,51,117,...,0.000000,2,0.042553,26,0,222,122,392,24,118
2,Benign,Benign,0,88,8,48.300000,1932,288.225000,45,100,...,0.000000,2,0.050000,26,0,222,120,395,27,118
3,Benign,Benign,0,88,8,45.156250,1445,264.281250,36,83,...,0.000000,2,0.062500,26,0,222,120,395,27,118
4,Benign,Benign,0,87,8,49.214286,2067,281.333333,45,103,...,0.086957,6,0.130435,26,0,222,124,392,24,118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58591,Ransomware-Shade-fa03be3078d1b9840f06745f160eb...,Malware,0,86,8,39.270270,1453,215.486487,40,92,...,0.026316,3,0.078947,26,0,221,120,389,24,116
58592,Ransomware-Shade-f56687137caf9a67678cde91e4614...,Malware,0,88,8,36.405405,1347,190.216216,39,91,...,0.000000,2,0.054054,26,0,221,116,389,24,116
58593,Ransomware-Shade-faddeea111a25da4d0888f3044ae9...,Malware,0,88,8,38.105263,1448,215.729730,40,92,...,0.050000,4,0.100000,26,0,221,120,389,24,116
58594,Ransomware-Shade-f866c086af2e1d8ebaa6f2c863157...,Malware,0,87,8,39.243243,1452,215.513513,40,92,...,0.000000,2,0.054054,26,0,221,120,389,24,116


In [4]:
X = X.drop('callbacks.nanonymous', axis=1)
X = X.drop('callbacks.ngeneric', axis=1)
X = X.drop('psxview.not_in_csrss_handles', axis=1)
X = X.drop('psxview.not_in_csrss_handles_false_avg', axis=1)
X = X.drop('psxview.not_in_deskthrd', axis=1)
X = X.drop('psxview.not_in_deskthrd_false_avg', axis=1)
X = X.drop('psxview.not_in_eprocess_pool', axis=1)
X = X.drop('psxview.not_in_eprocess_pool_false_avg', axis=1)
X = X.drop('psxview.not_in_ethread_pool', axis=1)
X = X.drop('psxview.not_in_ethread_pool_false_avg', axis=1)
X = X.drop('psxview.not_in_pslist', axis=1)
X = X.drop('psxview.not_in_pslist_false_avg', axis=1)
X = X.drop('psxview.not_in_pspcid_list', axis=1)
X = X.drop('psxview.not_in_pspcid_list_false_avg', axis=1)
X = X.drop('psxview.not_in_session', axis=1)
X = X.drop('psxview.not_in_session_false_avg', axis=1)
X = X.drop('svcscan.interactive_process_services', axis=1)

X = X.drop('Category', axis=1)
class_map = {'Benign':0, 'Malware':1}
X['Class'] = X['Class'].map(class_map)
Y = X['Class']
X = X.drop('Class', axis=1)
X = X.sort_index(axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

In [5]:
clf_tree = tree.DecisionTreeClassifier()
clf_tree.fit(X_train, y_train)

DecisionTreeClassifier()

In [6]:
y_pred = clf_tree.predict(X_test)

In [7]:
bucket_name = 'malmem-dataset'
blob_name = 'cart_model.pkl'

storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(blob_name)

blob.upload_from_string(pickle.dumps(clf_tree))

In [8]:
print(accuracy_score(y_test, y_pred))
print(average_precision_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(recall_score(y_test, y_pred))

0.9998293418283178
0.9997657577319421
0.999825975984686
0.9997679814385151
